# Pre-trained models and transfer learning

Training CNNs can take a lot of time, and a lot of data is required for that task. However, much of the time is spent to learn the best low-level filters that a network is using to extract patterns from images. A natural question arises - can we use a neural network trained on one dataset and adapt it to classifying different images without full training process?

This approach is called **transfer learning**, because we transfer some knowledge from one neural network model to another. In transfer learning, we typically start with a pre-trained model, which has been trained on some large image dataset, such as **ImageNet**. Those models can already do a good job extracting different features from generic images, and in many cases just building a classifier on top of those extracted features can yield a good result.

In [4]:
%pip install torch
%pip install torchvision
%pip install matplotlib
%pip install torchinfo
%pip install numpy
%pip install pytorchcv

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
import os
import pytorchcv

from pytorchcv import train, plot_results, display_dataset, train_long, check_image_dir


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "C:\Users\yena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\lib\shm.dll" or one of its dependencies.

## Cats vs. Dogs Dataset

In this unit, we will solve a real-life problem of classifying images of cats and dogs. For this reason, we will use [Kaggle Cats vs. Dogs Dataset](https://www.kaggle.com/c/dogs-vs-cats), which can also be downloaded [from Microsoft](https://www.microsoft.com/en-us/download/details.aspx?id=54765).

Let's download this dataset and extract it into `data` directory (this process may take some time!):

In [5]:
if not os.path.exists('./kagglecatsanddogs_5340.zip'):
    !wget -P data https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

NameError: name 'os' is not defined

In [ ]:
import zipfile
if not os.path.exists('data/PetImages'):
    with zipfile.ZipFile('./kagglecatsanddogs_5340.zip', 'r') as zip_ref:
        zip_ref.extractall('data')

Unfortunately, there are some corrupt image files in the dataset. We need to do quick cleaning to check for corrupted files. In order not to clobber this tutorial, we moved the code to verify dataset into a module.

In [ ]:
check_image_dir('data/PetImages/Cat/*.jpg')
check_image_dir('data/PetImages/Dog/*.jpg')

Next, let's load the images into PyTorch dataset, converting them to tensors and doing some normalization. We will apply `std_normalize` transform to bring images to the range expected by pre-trained VGG network:

In [ ]:
std_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
# 위  코드는 정규화 변환 기능을 생성 
# 정규화는 픽셀 강도 값의 범위를 변경하는 일반적인 전처리 단계
# 평균 및 표준편차 값은 일반적으로 ImageNet 데이터 세트에서 계산되며 여기에서 이미지의 각 채널(RGB)을 정규화하는 데 사용
# 각 채널에서 평균값을 빼고 그 결과를 각 채널의 표준 편차로 나누어 입력 전반에 걸쳐 일관된 스케일을 유지함으로써 훈련 중 수렴을 가속화함 
trans = transforms.Compose([      # 이 기능은 여러 이미지 변환을 단일 작업으로 결합하는ㅔ 데이터 전처리를 간소화하는 데 유용
        transforms.Resize(256),        # 입력 이미지의 크기를 256x256픽셀로 조정하는데  이 작업은 자르기를 작업 전에 수행
        transforms.CenterCrop(224), # 이미지를 중앙으로부터 224x224픽셀로 줄이는데 이 크기는 사전 학습된 많은 모델이 이 입력 크기를 예상하기 때문에 일반적으로 사용
        transforms.ToTensor(),           # PIL 이미지인 이미지를 PyTorch 텐서로 변환하 이미지의 픽셀 강도 값을 [0, 1] 범위로 스케일
        std_normalize])                     # 앞서 정의한 정규화를 적용

dataset = torchvision.datasets.ImageFolder('data/PetImages',transform=trans)
# 폴더의 구조가 클래스별로 구성되어 있다고 가정하여 폴더에서 데이터 세트를 로드
# 데이터/PetImages 내의 각 폴더 이름은 클래스를 나타내며 해당 클래스에 속하는 이미지가 포함되어 있는데 각 이미지에 트랜스 변환을 적용
trainset, testset = torch.utils.data.random_split(dataset,[20000,len(dataset)-20000])
# 이 함수는 데이터 집합을 두 개의 하위 집합으로 무작위로 분할
# 20,000개의 이미지가 포함된 훈련 세트와 나머지 이미지가 포함된 테스트 세트를 생성
# 분할을 통해 훈련 및 테스트 데이터 세트 모두에 모든 클래스가 무작위로 포함

display_dataset(dataset) # 데이터 세트의 일부 샘플을 시각화하여 전처리 및 데이터 로드 단계를 시각적으로 확인

## Pre-trained models

There are many different pre-trained models available inside `torchvision` module, and even more models can be found on the Internet. Let's see how simplest VGG-16 model can be loaded and used:

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

vgg = torchvision.models.vgg16(pretrained=True)
# ImageNet 데이터 세트에서 사전 학습된 VGG16 모델을 로드
# pretrained=True 매개변수는 모델 가중치가 ImageNet에서 학습한 후 그대로 로드되도록 함
#함VGG16은 이미지 분류 작업에서 효과적인 것으로 알려진 심층 컨볼루션 네트워크
sample_image = dataset[0][0].unsqueeze(0)
# dataset[0][0]: 데이터 세트에서 첫 번째 이미지를 추출하는데 ImageFolder 데이터 세트는 튜플(이미지, 레이블)을 반환하며, [0][0]은 첫 번째 튜플의 첫 번째 이미지에 액세스
# unsqueeze(0): 이 메서드는 텐서에 차원을 추가하여 모양을 [채널, 높이, 너비]에서 [batch_size, 채널, 높이, 너비]로 변환하는 데 사용하는데 PyTorch 모델은 일반적으로 하나의 이미지만 처리하는 경우에도 일괄 차원을 입력으로 기대
res = vgg(sample_image)
# 준비된 sample_image 텐서를 VGG16 모델에 전달
# 이 모델은 입력 이미지에 대한 1,000개의 ImageNet 클래스 각각에 대한 로그를 출력
print(res[0].argmax())
# 모델은 예측 배치를 반환하는데 샘플이 하나뿐이므로 res[0]은 첫 번째(그리고 유일한) 이미지의 예측에 액세스
# argmax() 함수는 이미지넷 클래스에서 예측된 클래스 레이블에 해당하는 가장 높은 로짓 값의 인덱스를 찾음

The result that we have received is a number of an `ImageNet` class, which can be looked up [here](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a). We can use the following code to automatically load this class table and return the result:

In [ ]:
import json, requests
# json: JSON 데이터를 Python 객체로 또는 그 반대로 구문 분석하는 모듈
# requests: HTTP 요청을 만드는 데  사용되는 Python 라이브러리인데 원격 URL에서 데이터를 가져오는 데 사용
class_map = json.loads(requests.get("https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json").text)
# requests.get(...): 이 함수는 지정된 URL(이 경우 ImageNet 클래스 인덱스와 레이블이 포함된 JSON 파일에 대한 링크)로 HTTP GET 요청을 하는데 requests.get의 응답에는 HTTP 응답에 대한 다양한 세부 정보가 포함되며 .text 속성은 응답의 본문을 문자열(JSON 형식이어야 함)로 가져옮
# json.loads(...): 이 함수는 JSON 형식의 문자열을 구문 분석하여 파이썬 사전으로 변환하는데 결과는 class_map에 저장
class_map = { int(k) : v for k,v in class_map.items() }
# dictionary comprehension을 사용하여 원래 class_map 사전을 처리하는데 키를 문자열에서 정수로 변환
# class_map.items(): 이 메서드는 사전의 항목(키-값 쌍)에 대한 뷰를 반환
#int(k): 각 키 k(원래 문자열)를 정수로 변환
# 그 결과 키가 정수인 새 사전이 생성되는데 모델 출력의 숫자 인덱스로 작업할 때 더 유용
class_map[res[0].argmax().item()]
# 신경망 모델의 예측을 기반으로 클래스 레이블을 검색
# res: 신경망의 출력 로짓을 포함하는 텐서
# res[0].argmax(): 로그 텐서에서 가장 높은 값의 인덱스를 찾는데 이 인덱스는 예측된 클래스에 해당하고 res에 배치 차원이 포함된 경우(배치 크기가 1인 경우에도) [0]이 필요
# item(): 파이토치 텐서에서 파이썬 정수로 결과를 변환
# class_map[...]: 정수 인덱스를 사용하여 class_map 딕셔너리에서 해당 클래스 레이블을 찾음

Let's also see the architecture of the VGG-16 network:

In [ ]:
summary(vgg,input_size=(1,3,224,224))
# 토치인포 라이브러리를 사용하여 VGG16 모델에 대한 자세한 요약을 제공
# summary() 함수는 PyTorch 모델에 대한 자세한 인사이트를 제공
# 요약에는 레이어의 수와 이름, 각 레이어의 출력 크기, 매개변수 수(훈련 가능 및 훈련 불가능) 등의 정보를 포함
# vgg: PyTorch의 torchvision.models에서 로드된 VGG16 모델의 인스턴스
# input_size=(1, 3, 224, 224): 모델이 예상하는 입력 텐서의 크기를 지정
# 1: 배치 크기 - 네트워크를 통해 한 번에 전달할 이미지 수입니다. 이 경우 1로 설정되어 있으므로 단일 이미지 입력에 대해 요약이 생성됨
# 3: 입력 이미지의 채널 수로, 표준 RGB 이미지에 해당
# 224, 224: 입력 이미지의 높이와 너비입니다. 이미지넷 챌린지를 위해 설계된 VGG16은 일반적으로 224x224픽셀 크기의 이미지를 예상
# 각 레이어에 다음과 같은 세부 정보가 나열됨
# 레이어 유형: 컨볼루션 레이어, 활성화 레이어(예: ReLU), 풀링 레이어, 완전히 연결된 레이어
# 출력 크기: 이미지가 네트워크를 통과할 때 이미지 크기와 채널 수가 어떻게 변환되는지 이해하는 데 도움이 되는 각 레이어의 출력 크기
# 매개변수 수: 각 레이어에는 매개변수가 카운트되어 총 매개변수와 학습 가능한 매개변수 수를 모두 표시하는데 모델의 복잡성과 용량을 이해하는 데 매우 중요

우리가 이미 알고 있는 레이어 외에도 **드롭아웃**이라는 또 다른 레이어 유형이 있음
이 레이어는 **정규화** 기법으로 작동하는데 정규화는 학습 알고리즘에 약간의 수정을 가하여 모델이 더 잘 일반화되도록 함
훈련 중에 드롭아웃 레이어는 이전 레이어에 있는 뉴런의 일부(약 30%)를 폐기하고, 그 뉴런 없이 훈련을 진행하는데 이렇게 하면 최적화 프로세스가 로컬 최소값에서 벗어나고 서로 다른 신경 경로 간에 결정적인 힘을 분배하여 네트워크의 전반적인 안정성을 향상시키는 데 도움이 됨

## GPU computations

Deep neural networks, such as VGG-16 and other more modern architectures require quite a lot of computational power to run. It makes sense to use GPU acceleration, if it is available. In order to do so, we need to explicitly move all tensors involved in the computation to GPU.

The way it is normally done is to check the availability of GPU in the code, and define `device` variable that points to the computational device - either GPU or CPU.


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Doing computations on device = {}'.format(device))

vgg.to(device)
sample_image = sample_image.to(device)

vgg(sample_image).argmax()


## Extracting VGG features

VGG-16을 사용하여 이미지에서 특징을 추출하려면 최종 분류 레이어가 없는 모델이 필요한데 "특징 추출기"는 `vgg.features` 메서드를 사용하여 얻을 수 있음:

In [ ]:
res = vgg.features(sample_image).cpu()
# vgg.features(sample_image): 모든 컨볼루션 및 풀링 레이어로 구성된 VGG16 모델의 특징 부분만 사용하는데 모델 끝에 완전히 연결된 레이어는 포함되지 않고 샘플_이미지는 이러한 레이어를 통과하여 특징 맵을 생성
# cpu(): 결과 텐서(피처 맵)를 CPU로 이동하는데 이 단계는 텐서가 원래 GPU에 있는 경우 필요합니다(샘플_이미지가 'cuda'일 수 있는 장치로 전송되었기 때문에). 이 단계는 CUDA 텐서를 지원하지 않을 수 있는 후속 작업이나 시각화 또는 추가 CPU 전용 처리를 준비할 때 호환성을 보장
plt.figure(figsize=(15,3))
# matplotlib로 플로팅할 새 그림을 만드는데 figsize=(15,3) 인수는 그림의 치수를 인치 단위로 설정하여 재구성된 피처 맵을 표시하는 데 적합한 넓고 짧은 영역을 제공
plt.imshow(res.detach().view(512,-1).T)
# res.detach(): 현재 계산 그래프에서 결과 텐서를 분리하는데 이 텐서에 대한 후속 연산에는 그라데이션 계산이 필요하지 않으므로 데이터 시각화 및 추론과 같은 연산에 유용
# view(512,-1): 텐서의 모양을 변경합니다. 512는 VGG16의 마지막 컨볼루션 레이어 출력에서의 특징 채널의 수이고 -1은 다른 차원을 기준으로 텐서에서 동일한 양의 데이터를 유지하는 데 필요한 크기를 계산하도록 PyTorch에 지시
# T: 행과 열을 바꾸어 재구성된 텐서를 전치하는데 데이터 레이아웃을 사람의 시각적 기대치에 더 잘 맞추기 위한 시각화의 일반적인 단계
# plt.imshow(): 텐서에서 이미지를 표시하는데 피처 맵이 단일 이미지로 시각화되며, 각 행은 피처 채널을 나타내고 열은 해당 채널에서 집계된 공간 데이터를 나타냄
print(res.size())
# res.size(): 텐서 res의 크기를 출력하는데 이 텐서는 컨볼루션 레이어의 출력을 나타냄
# 입력 이미지 크기가 224x224인 일반적인 VGG16 모델의 경우, 마지막 컨볼루션 레이어 바로 뒤(그리고 완전히 연결된 레이어 이전)의 텐서 크기는 [1, 512, 7, 7]을 나타냄
# 1 배치 크기(처음에 샘플_이미지에 배치 차원을 추가했기 때문에)
# 512개의 피처 채널
# 피처 맵의 7x7 공간 차원

특징 텐서의 크기는 512x7x7이지만 이를 시각화하기 위해 2D 형태로 재구성해야 함

이러한 특징을 이용해 이미지를 분류할 수 있는지 확인해 보면 이미지의 일부(이 경우 800개)를 수동으로 가져와서 특징 벡터를 미리 계산한 결과를 'feature_tensor'라는 하나의 큰 텐서에 저장하고 레이블도 'label_tensor'에 저장

In [ ]:
# VGG16 모델을 사용해 데이터 세트를 처리하여 컨볼루션 레이어에서 특징 표현을 추출하고 해당 레이블을 수집하는 방법을 설명
# 이는 사전 학습된 네트워크의 심층 특징을 추가 분석, 더 간단한 모델의 학습 또는 전이 학습 시나리오에 사용하는 특징 추출 작업의 일반적인 관행

bs = 8
dl = torch.utils.data.DataLoader(dataset,batch_size=bs,shuffle=True)
num = bs*100
# bs = 8: 배치 크기를 8로 설정하여 루프 중에 한 번에 처리할 이미지 수를 결정
# 데이터로더: 지정된 배치 크기와 셔플링이 활성화된 데이터 로더를 초기화하는데 셔플링은 훈련 중 편차를 줄이는 데 도움이 되며 모델이 데이터의 순서에 대해 특정 내용을 학습하지 않도록 함
# num = bs * 100: 처리할 총 이미지 수를 설정합니다(이 경우 800개). 이는 데이터 세트의 하위 집합으로 처리를 제한하거나 메모리 제약 조건에 맞출 때 유용

feature_tensor = torch.zeros(num,512*7*7).to(device)
label_tensor = torch.zeros(num).to(device)
# 추출된 특징과 해당 레이블을 각각 저장하기 위해서 두 개의 텐서 feature_tensor와 label_tensor를 초기화
# feature_tensor는 800개의 샘플에 대해 평탄화된 특징을 보유하도록 크기가 조정되며, 각 특징 벡터에는 512*7*7 요소가 있음(vgg.features의 출력에서 7x7 특징 맵의 512개 채널에서)
# 두 텐서 모두 지정된 장치로 이동되며, 가능한 경우 GPU가 될 수 있음

i = 0
for x,l in dl:
    with torch.no_grad():
        f = vgg.features(x.to(device))
        feature_tensor[i:i+bs] = f.view(bs,-1)
        label_tensor[i:i+bs] = l
        i+=bs
        print('.',end='')
        if i>=num:
            break

# 루프는 데이터로더 dl을 사용하여 데이터 집합을 반복하는데 각 반복은 이미지 x와 그에 해당하는 레이블 l의 배치를 생성
# with torch.no_grad():다: 작업 중에 그라데이션을 계산하지 않도록 하여 추론이나 특징 추출에 그라데이션이 필요하지 않으므로 메모리와 계산을 절약할 수 있음
# vgg.features(x.to(device)): 각 배치를 디바이스(GPU 또는 CPU)로 이동하고 VGG 모델의 특징 레이어를 통과시켜 처리
# f.view(bs, -1): 모델에서 출력 피처 텐서를 재구성하여 피처를 이미지당 단일 벡터로 평평하게 만듦
# 특징 텐서와 라벨 텐서는 현재 배치의 데이터로 업데이트
# 루프는 간단한 진행률 표시기로 처리된 각 배치에 대해 점을 인쇄
# num에 도달하거나 초과하면 루프가 중단되어 정확히 800개의 이미지가 처리


이 텐서에서 데이터를 가져오는 `vgg_dataset`을 정의하고 `random_split` 함수를 사용하여 학습 세트와 테스트 세트로 분할한 다음 추출된 특징 위에 작은 1계층 고밀도 분류기 네트워크를 학습할 수 있음

In [ ]:
# VGG16 모델에서 추출한 특징을 사용하여 간단한 신경망을 훈련시키는 과정을 보여

# TensorDataset 생성
vgg_dataset = torch.utils.data.TensorDataset(feature_tensor,label_tensor.to(torch.long))
# feature_tensor는 VGG16 모델로부터 추출된 특징을 담고 있으며, label_tensor는 각 특징에 해당하는 레이블을 담고 있음
# label_tensor.to(torch.long)을 통해 레이블 데이터 타입을 long 형태로 변경하는데 이는 분류 작업에서 필요
# TensorDataset은 주어진 텐서들을 하나의 데이터셋으로 묶어줌

# 데이터셋 분할
train_ds, test_ds = torch.utils.data.random_split(vgg_dataset,[700,100])
# vgg_dataset을 훈련 데이터셋 700개와 테스트 데이터셋 100개로 무작위 분할

# DataLoader 설정
train_loader = torch.utils.data.DataLoader(train_ds,batch_size=32)
test_loader = torch.utils.data.DataLoader(test_ds,batch_size=32)
# DataLoader는 배치 크기를 32로 설정하여 훈련 및 테스트 데이터셋을 처리할 수 있도록 준비
# 배치 처리를 통해 데이터를 효율적으로 로딩하고 모델 훈련 시간을 단축할 수 있음

# 신경망 정의
net = torch.nn.Sequential(torch.nn.Linear(512*7*7,2),torch.nn.LogSoftmax()).to(device)
# torch.nn.Sequential을 사용해 신경망을 구성하는데 VGG16의 특징을 입력으로 받아 두 개의 클래스로 분류하는 선형 계층(Linear)과 로그 소프트맥스 계층(LogSoftmax)을 포함
# 모델을 device (GPU 또는 CPU)로 이동

# 모델훈련
history = train(net,train_loader,test_loader)
# 정의된 train 함수를 사용하여 모델을 훈련하는데 이 함수는 train_loader로부터 데이터를 받아 모델을 학습하고, test_loader를 사용하여 모델을 평가
# history는 훈련 과정에서의 로스(loss)나 정확도(accuracy)와 같은 결과를 저장할 수 있음

결과는 좋은데 거의 96%의 확률로 고양이와 개를 구별할 수 있지만 수동 특징 추출에는 많은 시간이 걸리기 때문에 전체 이미지 중 일부에만 테스트 했음

## Transfer learning using one VGG network

훈련 중에 원래의 VGG-16 네트워크를 전체적으로 사용하여 기능을 수동으로 사전 계산하는 것을 피할 수 있는데 VGG-16 객체 구조를 살펴보면 다음과 같음

In [ ]:
print(vgg)

네트워크에 포함된 것은 다음과 같음:
* 다수의 컨볼루션 및 풀링 레이어로 구성된 특징 추출기(`features`)
* 평균 풀링 레이어(`avgpool`)
* 여러 개의 고밀도 레이어로 구성된 최종 `분류기`로, 25,088개의 입력 특징을 1,000개의 클래스(ImageNet의 클래스 수)로 변환합니다.

데이터 세트를 분류할 엔드 투 엔드 모델을 훈련하려면 다음과 같이 해야 합니다:
* 최종 분류기를 필요한 클래스 수를 생성할 수 있는 분류기로 교체해야 하는데 이번 경우, 25,088개의 입력과 2개의 출력 뉴런이 있는 하나의 '선형' 레이어를 사용할 수 있음
* 컨볼루션 특징 추출기의 가중치를 동결하여 학습되지 않도록 하는데 학습되지 않은 분류자 레이어는 컨볼루션 추출기의 원래 사전 학습된 가중치를 변경할 수 있으므로 처음에 이 동결을 수행하는 것이 좋음
가중치 고정은 모든 파라미터의 `requires_grad` 속성을 `False`로 설정하여 수행할 수 있음

In [ ]:
# 사전 훈련된 VGG 모델을 수정하고, 그 구조를 확인하는 과정

# 분류기 교체
vgg.classifier = torch.nn.Linear(25088,2).to(device)
# vgg.classifier를 새로운 선형 레이어(torch.nn.Linear)로 교체하여, 원래의 VGG 모델을 두 개의 클래스로 분류하는 작업에 적합하도록 설정하는데 이 레이어는 입력 특성 25088개를 받아 두 클래스로 예측
# .to(device)를 사용하여 모델을 적절한 장치(GPU 또는 CPU)로 이동

# 특성 추출기의 그라디언트 업데이트 비활성화
for x in vgg.features.parameters():
    x.requires_grad = False
# vgg.features의 모든 파라미터에 대해 그라디언트 업데이트를 비활성화하여, 사전 훈련된 특성 추출기 부분이 학습 과정에서 변경되지 않도록 하는데 이는 전이 학습 시 컴퓨팅 자원을 절약하고 오버피팅을 방지하는 데 도움이 됨

# 모델 요약 출력
summary(vgg,(1, 3,244,244))
# summary() 함수를 사용하여 수정된 VGG 모델의 구조를 확인할 수 있는데 이 함수는 모델이 처리할 수 있는 입력 크기(1개 배치, 3개 채널, 244x244 픽셀)를 기준으로 각 레이어의 출력 크기와 파라미터 수를 보여


요약에서 볼 수 있듯이 이 모델에는 약 1,500만 개의 파라미터가 포함되어 있지만, 그 중 학습 가능한 것은 분류 계층의 가중치인 5만 개뿐인데 이는 더 적은 수의 예제로 더 적은 수의 파라미터를 미세 조정할 수 있기 때문에 이는 좋음

다음은 원본 데이터 세트를 사용하여 모델을 훈련하는 과정인데 이 것은 시간이 오래 걸리므로 에포크가 끝날 때까지 기다리지 않고 중간 결과를 출력하는 `train_long` 함수를 사용 

In [ ]:
# 데이터셋을 훈련 세트와 테스트 세트로 분할하고, 이를 로딩하기 위한 데이터 로더를 설정한 후, VGG 모델을 훈련시키는 과정

# 데이터셋 분할
trainset, testset = torch.utils.data.random_split(dataset,[20000,len(dataset)-20000])
#전체 데이터셋을 20,000개의 훈련 세트와 나머지를 테스트 세트로 무작위 분할

# 데이터 로더 설정
train_loader = torch.utils.data.DataLoader(trainset,batch_size=16)
test_loader = torch.utils.data.DataLoader(testset,batch_size=16)
# 훈련 세트와 테스트 세트를 각각 배치 크기 16으로 로드하기 위한 데이터 로더를 생성

# 모델 훈련
train_long(vgg,train_loader,test_loader,loss_fn=torch.nn.CrossEntropyLoss(),epochs=1,print_freq=90)
# train_long 함수를 사용하여 VGG 모델을 훈련하는데 loss_fn은 교차 엔트로피 손실 함수를 사용하고, 훈련은 1 에포크 동안 진행
# print_freq=90은 훈련 과정 중 로그를 90번째 배치마다 출력하도록 설정


상당히 정확한 고양이 대 개 분류기를 얻었는데 나중에 사용할 수 있도록 저장

In [ ]:
# VGG 모델을 하드 드라이브에 저장하여 나중에 다시 불러오거나 다른 시스템으로 전송할 수 있게 하는데 모델을 저장하는 것은 학습 후에 모델의 상태를 보존하고, 필요할 때 언제든지 모델을 재사용할 수 있는 유용한 방법
torch.save(vgg,'data/cats_dogs.pth')
# torch.save: 이 함수는 PyTorch 모델이나 텐서를 지정된 파일에 저장하는데 vgg 모델이 data/cats_dogs.pth 파일에 저장
# 'data/cats_dogs.pth': 모델이 저장될 파일의 경로이고 .pth 확장자는 PyTorch 모델 파일을 의미

언제든지 파일에서 모델을 로드할 수 있는데 다음 실험에서 모델이 변경되는 경우에도 처음부터 다시 시작할 필요가 없으므로 유용

In [ ]:
# 학습된 모델을 불러와서 추가로 학습시키거나 평가를 수행할 때 사용할 수 있음
vgg = torch.load('data/cats_dogs.pth')
# torch.load: 이 함수는 파일에서 PyTorch 모델이나 텐서를 불러오는데 'data/cats_dogs.pth' 경로에서 저장된 VGG 모델을 불러와서 vgg 변수에 할당
# 'data/cats_dogs.pth': 모델이 저장된 파일의 경로


## Fine-tuning transfer learning

이전 섹션에서는 자체 데이터 세트의 이미지를 분류하기 위해 최종 분류 레이어를 훈련했지만 특징 추출기를 다시 훈련시키지 않았고, 모델이 ImageNet 데이터에서 학습한 특징에 의존했음
하지만 객체가 일반 ImageNet 이미지와 시각적으로 다른 경우 이러한 특징 조합이 가장 효과적이지 않을 수 있기때문에 컨볼루션 레이어도 학습을 시작하는 것이 좋음

이를 위해 이전에 고정했던 컨볼루션 필터 매개변수를 고정 해제

> **Note:** 분류 계층의 가중치를 안정화하려면 먼저 파라미터를 동결하고 여러 단계의 학습을 수행하는 것이 중요
고정되지 않은 파라미터로 엔드 투 엔드 네트워크 훈련을 즉시 시작하면 큰 오류로 인해 컨볼루션 계층에서 미리 훈련된 가중치가 파괴적으로 변경될 수 있음

In [ ]:
# 모델을 특정 작업에 맞게 미세 조정하거나 새로운 데이터셋에 대해 모델을 전체적으로 다시 학습시키고자 할 때 사용하는데 예를 들어, 사전 훈련된 모델을 새로운 이미지 분류 작업에 적용하면서 더 많은 계층을 조정하고 싶을 때 유용

for x in vgg.features.parameters():
    x.requires_grad = True

# vgg.features.parameters(): VGG 모델의 features 부분에서 모든 파라미터를 반복하는데 features 부분은 일반적으로 모델의 합성곱 계층들을 포함
# x.requires_grad = True: 각 파라미터 x에 대해, 그라디언트 계산을 활성화하고 파라미터가 학습 과정에서 업데이트 될 수 있도록 설정

고정을 해제하면 몇 회 더 훈련할 수 있는데 사전 학습된 가중치에 미치는 영향을 최소화하기 위해 학습 속도를 낮게 선택할 수도 있지만 학습 속도가 낮더라도 훈련 초기에는 정확도가 떨어지다가 고정 가중치의 경우보다 약간 더 높은 수준에 도달할 때까지 정확도가 떨어질 것으로 예상

> **Note:** 이 훈련은 네트워크의 여러 레이어를 통해 그라디언트를 다시 전파해야 하기 때문에 훨씬 느리게 진행

In [ ]:
# 모델의 훈련을 관리하고, 주어진 파라미터에 따라 반복적으로 데이터를 처리하며, 지정된 주기로 훈련 상태를 출력하는데 데이터 로더를 통해 자동으로 배치 처리되는 데이터를 받고, 각 에포크 후에 테스트 데이터셋으로 모델을 평가할 수 있음

train_long(vgg,train_loader,test_loader,loss_fn=torch.nn.CrossEntropyLoss(),epochs=1,print_freq=90,lr=0.0001)

# vgg: 훈련할 모델로, VGG 네트워크를 지정
# train_loader: 훈련 데이터를 배치 단위로 모델에 제공하는 데이터 로더
# test_loader: 테스트 데이터를 배치 단위로 모델에 제공하여 평가하는 데 사용하는 데이터 로더
# loss_fn=torch.nn.CrossEntropyLoss(): 손실 함수로 교차 엔트로피 손실을 사용하는데 이 함수는 분류 문제에 자주 사용되며, 모델의 예측과 실제 레이블 사이의 오차를 계산
# epochs=1: 모델을 1 에포크 동안 훈련하는데 에포크는 전체 훈련 데이터셋이 모델을 한 번 통과하는 것을 의미
# print_freq=90: 훈련 과정에서 90번째 배치마다 로그를 출력하도록 설정하는데 이는 훈련의 진행 상황을 모니터링하는 데 유용
# lr=0.0001: 학습률을 0.0001로 설정하는데 학습률은 최적화 알고리즘이 손실 그라디언트에 따라 모델의 가중치를 얼마나 조정할지 결정하는 매개변수

## Other computer vision models

VGG-16은 가장 간단한 컴퓨터 비전 아키텍처 중 하나입니다. 토치비전` 패키지는 더 많은 사전 훈련된 네트워크를 제공하는데 그 중 가장 자주 사용되는 것은 Microsoft에서 개발한 **ResNet** 아키텍처와 Google의 **Inception** 아키텍처
예를 들어 가장 간단한 ResNet-18 모델의 아키텍처를 살펴보면 다음과 같음 (ResNet은 다양한 깊이를 가진 모델 제품군으로, 정말 깊은 모델이 어떤 모습인지 보려면 ResNet-151로 실험해 볼 수 있음)음

In [ ]:
# 모델의 구조를 확인하고자 할 때 사용되며, 특히 새로운 프로젝트를 시작할 때나 모델을 커스터마이징 하기 전에 기본 구조를 이해하는 데 도움을 줌

resnet = torchvision.models.resnet18()
print(resnet)

# torchvision.models.resnet18(): PyTorch의 torchvision 라이브러리 내에서 ResNet-18 모델을 불러오는데 ResNet-18은 18개의 레이어를 가진 컨볼루션 신경망으로, 깊은 신경망에서 자주 발생하는 손실 문제를 개선하기 위해 잔차 연결(residual connections)을 사용
# print(resnet): 이 명령은 불러온 ResNet-18 모델의 구조를 출력하는데 출력되는 모델 구조에는 다양한 컨볼루션 레이어, 배치 정규화 레이어, 활성화 함수, 풀링 레이어, 완전 연결 레이어 등이 포함되어 있으며, 이러한 각 구성 요소들이 어떻게 연결되어 있는지를 보여줌

이 모델에는 특징 추출기와 최종 분류기(`fc`)라는 동일한 구성 요소가 포함되어 있는데  전이 학습에 VGG-16을 사용했던 것과 똑같은 방식으로 이 모델을 사용할 수 있고 위의 코드를 다른 ResNet 모델을 기본 모델로 사용하여 실험해보고 정확도가 어떻게 변하는지 확인할 수 있음

## Batch Normalization

이 네트워크에는 또 다른 유형의 레이어가 포함되어 있음: **배치 정규화**의 개념은 신경망을 통해 전달되는 값을 적절한 간격으로 가져오는 것인데 일반적으로 신경망은 모든 값이 [-1,1] 또는 [0,1] 범위에 있을 때 가장 잘 작동하며, 이것이 바로 입력 데이터를 그에 따라 스케일링/노멀라이즈하는 이유
그러나 딥 네트워크를 훈련하는 동안 값이 이 범위를 크게 벗어나는 경우가 발생하여 훈련에 문제가 생길 수 있는데 배치 정규화 계층은 현재 미니 배치의 모든 값에 대한 평균과 표준 편차를 계산하고 이를 사용하여 신호를 신경망 계층으로 전달하기 전에 정규화하는 데 사용하고 이를 통해 심층 네트워크의 안정성이 크게 향상될 수 있음


## Takeaway

전이 학습을 사용하여 사용자 지정 객체 분류 작업을 위한 분류기를 빠르게 구성하고 높은 정확도를 달성할 수 있지만 이 예는 완전히 공정하지는 않았음 ... 원래 VGG-16 네트워크는 고양이와 개를 인식하도록 사전 학습되었기 때문에 네트워크에 이미 존재하는 대부분의 패턴을 재사용했을 뿐이었기 때문이고 공장의 생산 라인에 대한 세부 정보나 다른 나뭇잎과 같이 보다 이질적인 도메인별 객체에 대해서는 정확도가 낮을 것으로 예상할 수 있음

지금 우리가 풀고 있는 더 복잡한 작업은 더 높은 연산 능력이 필요하며 CPU로는 쉽게 풀 수 없다는 것을 알 수 있는데 다음 단원에서는 더 가벼운 구현을 사용하여 더 낮은 컴퓨팅 리소스를 사용하여 동일한 모델을 훈련하여 정확도를 약간만 낮추는 방법을 시도할 계획